In [239]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [240]:
dataset = pd.read_csv('titanic.csv')
dataset.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [241]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


#### Dropping Columns which are not useful

In [242]:
columns = ['PassengerId', 'Name', 'Ticket', 'Cabin']
dataset.drop(columns, axis=1, inplace=True)

In [243]:
dataset.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [244]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


#### we can observe that Age, Cabin, Embarked have missing values in them

In [245]:
dataset['Age'] = dataset['Age'].interpolate()

In [246]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [247]:
dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace=True)

In [248]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


### Outlier Detection

In [249]:
def outlier_thresholds(df, col_name, q1=0.25, q3=0.75):
    quartile1 = df[col_name].quantile(q1)
    quartile3 = df[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    # print(df[(df["Fare"] < low_limit) | (df["Fare"] > up_limit)].head())
    return low_limit, up_limit

In [250]:
numerical_cols = dataset.select_dtypes(include=[float, int]).columns.tolist()

def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False
    
cols_with_outliers = []
for i in numerical_cols:
    if check_outlier(dataset, i):
        cols_with_outliers.append(i)
print(cols_with_outliers)

['Age', 'SibSp', 'Parch', 'Fare']


### Now we will assign outliers with thresholds.

In [251]:
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

for i in cols_with_outliers:
    replace_with_thresholds(dataset, i)


In [252]:
cols_with_outliers2 = []
for i in numerical_cols:
    if check_outlier(dataset, i):
        cols_with_outliers2.append(i)
print(cols_with_outliers2)

[]


- so, all the outliers are removed

#### Now lets deal with Categorical Values and convert them to numerical 

In [253]:
dataset.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1.0,0,7.2500,S
1,1,1,female,38.0,1.0,0,65.6344,C
2,1,3,female,26.0,0.0,0,7.9250,S
3,1,1,female,35.0,1.0,0,53.1000,S
4,0,3,male,35.0,0.0,0,8.0500,S


In [254]:
dummies = []
cols = ['Pclass', 'Sex', 'Embarked']
for col in cols:
    dummies.append(pd.get_dummies(dataset[col]))
titanic_dummies = pd.concat(dummies, axis=1)
titanic_dummies

,1,2,3,female,male,C,Q,S
0,0,0,1,0,1,0,0,1
1,1,0,0,1,0,1,0,0
2,0,0,1,1,0,0,0,1
3,1,0,0,1,0,0,0,1
4,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...
886,0,1,0,0,1,0,0,1
887,1,0,0,1,0,0,0,1
888,0,0,1,1,0,0,0,1
889,1,0,0,0,1,1,0,0


In [255]:
df = pd.concat((dataset, titanic_dummies),axis=1)

In [256]:
df.drop(cols, axis=1,inplace=True)

In [257]:
df.head()

,Survived,Age,SibSp,Parch,Fare,1,2,3,female,male,C,Q,S
0,0,22.0,1.0,0,7.2500,0,0,1,0,1,0,0,1
1,1,38.0,1.0,0,65.6344,1,0,0,1,0,1,0,0
2,1,26.0,0.0,0,7.9250,0,0,1,1,0,0,0,1
3,1,35.0,1.0,0,53.1000,1,0,0,1,0,0,0,1
4,0,35.0,0.0,0,8.0500,0,0,1,0,1,0,0,1


#### Dividing dataset into independent(X) and dependent(y) variabls

In [258]:
X = df.drop('Survived',axis=1).values
y = df['Survived'].values

In [259]:
X

array([[22. ,  1. ,  0. , ...,  0. ,  0. ,  1. ],
       [38. ,  1. ,  0. , ...,  1. ,  0. ,  0. ],
       [26. ,  0. ,  0. , ...,  0. ,  0. ,  1. ],
       ...,
       [22.5,  1. ,  0. , ...,  0. ,  0. ,  1. ],
       [26. ,  0. ,  0. , ...,  1. ,  0. ,  0. ],
       [32. ,  0. ,  0. , ...,  0. ,  1. ,  0. ]])

In [260]:
y

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,

#### Train Test Split of Dataset

In [261]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 12), (179, 12), (712,), (179,))

#### Standardization of data

In [262]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [263]:
X_train_scaled

array([[ 1.18233166, -0.60470133,  0.        , ..., -0.46146201,
        -0.30335547,  0.59248936],
       [-0.4734317 , -0.60470133,  0.        , ..., -0.46146201,
        -0.30335547,  0.59248936],
       [ 0.18887364, -0.60470133,  0.        , ..., -0.46146201,
        -0.30335547,  0.59248936],
       ...,
       [ 0.85117899,  2.15080169,  0.        , ..., -0.46146201,
        -0.30335547,  0.59248936],
       [-1.13573705,  0.77305018,  0.        , ..., -0.46146201,
        -0.30335547,  0.59248936],
       [-0.62061067, -0.60470133,  0.        , ..., -0.46146201,
        -0.30335547,  0.59248936]])

In [264]:
X_test_scaled

array([[-0.4734317 ,  0.77305018,  0.        , ...,  2.16702561,
        -0.30335547, -1.68779402],
       [ 0.11528416, -0.60470133,  0.        , ..., -0.46146201,
        -0.30335547,  0.59248936],
       [-0.69420015, -0.60470133,  0.        , ..., -0.46146201,
        -0.30335547,  0.59248936],
       ...,
       [ 0.63041054,  0.77305018,  0.        , ..., -0.46146201,
        -0.30335547,  0.59248936],
       [-0.9149686 , -0.60470133,  0.        , ..., -0.46146201,
        -0.30335547,  0.59248936],
       [-1.87163188,  0.77305018,  0.        , ..., -0.46146201,
        -0.30335547,  0.59248936]])

#### Apply Principal Component Analysis to find best features

In [265]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(X_train_scaled)
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)
cumulative_variance_ratio

array([0.25094991, 0.43235196, 0.60522207, 0.72981847, 0.83604219,
       0.92643368, 0.98320152, 1.        , 1.        , 1.        ,
       1.        , 1.        ])

- We can see that only first 7 compenents captures almost all variance of dataset so we will take first 7 principal component

In [266]:

pca = PCA(n_components=7)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [267]:
X_train_pca.shape, X_test_pca.shape, y_train.shape, y_test.shape

((712, 7), (179, 7), (712,), (179,))

### Now lastly lets use Logistic Regression Algorithm to make our predictive model

In [268]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train_pca, y_train)
y_pred = clf.predict(X_test_pca)

In [269]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(accuracy_score(y_pred=y_pred, y_true=y_test))
print(confusion_matrix(y_pred=y_pred, y_true=y_test))

0.7877094972067039
[[88 17]
 [21 53]]
